In [1]:
import pandas as pd
import pyLDAvis
import pyLDAvis.lda_model

C:\Users\USER\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from sklearn.feature_extraction.text import CountVectorizer   #gensim이 아닌사이킷-런을 사용해서도 LDA를 수행할 수 있다!
from sklearn.decomposition import LatentDirichletAllocation

## 데이터 불러오기

In [3]:
df_광주 = pd.read_csv('data/finish/광주.csv')
df_대구 = pd.read_csv('data/finish/대구.csv')
df_대전 = pd.read_csv('data/finish/대전.csv')
df_부산 = pd.read_csv('data/finish/부산.csv')
df_서울 = pd.read_csv('data/finish/서울.csv')
df_울산 = pd.read_csv('data/finish/울산.csv')
df_인천 = pd.read_csv('data/finish/인천.csv')
df_제주 = pd.read_csv('data/finish/제주.csv')
df_대한민국 = pd.read_csv('data/finish/대한민국.csv')

## LDA  (from sklearn)

In [4]:
# 데이터프레임들을 리스트로 저장
dataframes = [df_광주, df_대구, df_대전, df_인천, df_울산, df_서울, df_제주, df_부산,
              df_대한민국]

merged_df = pd.concat([df[['preprocessed_data']] for df in dataframes], ignore_index=True)
merged_df

,preprocessed_data
0,민주 운동 연루 기소 유예 처분 재수 검찰 죄 판단 서울서부지검 형사 부부장 검사 ...
1,의회 본회의 추경안 제안 설명 제안 설명 강기정 광주시장 제공 판매 금지 연합뉴스 ...
2,복합 쇼핑몰 종 오월 정신 헌법 수록 복지 강화 추경 원 광주시의회 힘 지혜 뉴시스...
3,내일 일자리 박람회 포스터 제공 뉴스 뉴스 박준 기자 우수 기업 인재 일자리 박람회...
4,고용 노동청 주최 우수 중견 중소기업 개사 참여 내일 일자리 박람회 포스터 사진 제...
...,...
15252,남수단 최초 모내기 방식 적용 벼농사 시작 빛 부대 남수단 라이스 연합 뉴스 남수단...
15253,의왕시 공동 주관 훈련 태풍 집중호우 재난 상황 대비 체계 점검 역량 강화 이데일리...
15254,탄소 선도 유럽 시멘트 유럽 시멘트 노폐물 제거 정맥 산업 인식 순환 자원 활용 쓰...
15255,사진 출처 경기도 의왕시 오봉 산마을 단지 행복 주택 의왕시 공동 주관 재난 대응 ...


In [5]:
merged_df = merged_df.dropna()
merged_df

,preprocessed_data
0,민주 운동 연루 기소 유예 처분 재수 검찰 죄 판단 서울서부지검 형사 부부장 검사 ...
1,의회 본회의 추경안 제안 설명 제안 설명 강기정 광주시장 제공 판매 금지 연합뉴스 ...
2,복합 쇼핑몰 종 오월 정신 헌법 수록 복지 강화 추경 원 광주시의회 힘 지혜 뉴시스...
3,내일 일자리 박람회 포스터 제공 뉴스 뉴스 박준 기자 우수 기업 인재 일자리 박람회...
4,고용 노동청 주최 우수 중견 중소기업 개사 참여 내일 일자리 박람회 포스터 사진 제...
...,...
15252,남수단 최초 모내기 방식 적용 벼농사 시작 빛 부대 남수단 라이스 연합 뉴스 남수단...
15253,의왕시 공동 주관 훈련 태풍 집중호우 재난 상황 대비 체계 점검 역량 강화 이데일리...
15254,탄소 선도 유럽 시멘트 유럽 시멘트 노폐물 제거 정맥 산업 인식 순환 자원 활용 쓰...
15255,사진 출처 경기도 의왕시 오봉 산마을 단지 행복 주택 의왕시 공동 주관 재난 대응 ...


In [6]:
vectorizer = CountVectorizer()                                  #CountVectorizer를 사용하여 단어의 빈도에 대한 벡터 생성
X = vectorizer.fit_transform(merged_df['preprocessed_data'])

In [7]:
N_TOPICS = 9          #우리의 주제는 9개  #ppt에서 추가 6개 주제확인해본건 N_TOPICS = 6 으로 변경후 사용
lda_model = LatentDirichletAllocation(n_components=N_TOPICS, random_state=42)
lda_model.fit(X)

LatentDirichletAllocation(n_components=9, random_state=42)

In [8]:
def get_topic_ratio_for_each_document(lda_model, X):
    # 문서별 토픽 분포 계산
    doc_topic_distr = lda_model.transform(X)
    
    results = []
    for i, topic_dist in enumerate(doc_topic_distr):
        sorted_topic_list = sorted(enumerate(topic_dist), key=lambda x: x[1], reverse=True)
        most_important_topic, most_important_ratio = sorted_topic_list[0]
        doc_result = [i, most_important_topic, most_important_ratio, sorted_topic_list]
        results.append(doc_result)
    
    results_df = pd.DataFrame(results, columns=['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중'])
    return results_df

df_results = get_topic_ratio_for_each_document(lda_model, X)

In [10]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda_model, X, vectorizer, mds='tsne')
pyLDAvis.display(panel)

C:\Users\USER\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(
